In [1]:
import datetime as dt
import pickle

from hostess.subutils import Viewer
from yamcs.client import YamcsClient

In [2]:
of_interest = (
    '/ViperGround/Images/ImageData/Hazcam_back_left_icer',
    '/ViperGround/Images/ImageData/Hazcam_back_right_icer',
    '/ViperGround/Images/ImageData/Hazcam_front_left_icer',
    '/ViperGround/Images/ImageData/Hazcam_front_right_icer',
    "/ViperGround/Images/ImageData/Navcam_left_icer",
    "/ViperGround/Images/ImageData/Navcam_right_icer",
    "/ViperGround/Images/ImageData/Aftcam_left_icer",
    "/ViperGround/Images/ImageData/Aftcam_right_icer",
    "/ViperRover/LightsControl/state"
)

In [3]:
def unpack_parameter_value(value):
    rec = {}
    for key in (
        'eng_value',
        'generation_time',
        'monitoring_result',
        'name',
        'processing_status',
        'range_condition',
        'raw_value',
        'reception_time',
        'validity_duration',
        'validity_status'
    ):
        rec[key] = getattr(value, key)
    return rec

def unpack_parameters(messages):
    return [
        unpack_parameter_value(value) 
        for value in messages.parameters
    ]

In [4]:
server_process = Viewer.from_command(
    "build_9/viper-yamcs-9.0.0-SNAPSHOT-8db3782c13/viper-yamcs-9.0.0-SNAPSHOT/bin/yamcsd", 
    _bg=True,
)
yamcsd_url = 'localhost:8090/yamcs'
server_process.wait_for_output('err')

In [8]:
client = YamcsClient(yamcsd_url)
archive = client.get_archive('viper')

results = []
vals = archive.stream_parameter_values(
    of_interest,
    # unnecessarily large range but whatever
    start=dt.datetime(2023, 1, 1, 0, 0, 0, tzinfo=dt.timezone.utc),
    stop=dt.datetime(2024, 12, 15, 0, 0, 0, tzinfo=dt.timezone.utc)
)
for val in vals:
    results += unpack_parameters(val)
server_process.terminate()

In [11]:
results[0]['generation_time']

datetime.datetime(2024, 11, 13, 18, 19, 35, 346000, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=68400), 'EST'))

In [ ]:
with open('of_interest_23_10_04.pkl', 'wb') as stream:
    pickle.dump(results, stream)